In [44]:
import pandas as pd
import geopandas as gpd

from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper,
    LabelSet
)
from bokeh.plotting import figure, output_notebook, output_file, show

from bokeh.sampledata.us_counties import data as counties
from bokeh.sampledata.us_states import data as states

%matplotlib inline

In [45]:
output_notebook()

Loading BokehJS ...

In [13]:
states = {
    code: state for code, state in states.items() if (not state['name'] == 'Alaska' and 
                                                      not state['name'] == 'Hawaii' and 
                                                      not state['name'] == 'Puerto Rico')
}

state_xs = [state["lons"] for state in states.values()]
state_ys = [state["lats"] for state in states.values()]

In [2]:
permits = pd.read_csv('permits.csv')

In [7]:
permits

,state,StateAbbr,county,countyname,variable,year,value
0,1,AL,1,Autauga County,All Permits,2010,191
1,1,AL,1,Autauga County,All Permits,2009,110
2,1,AL,1,Autauga County,All Permits,2008,173
3,1,AL,1,Autauga County,All Permits,2007,260
4,1,AL,1,Autauga County,All Permits,2006,347
5,1,AL,1,Autauga County,All Permits,2005,313
6,1,AL,1,Autauga County,All Permits,2004,367
7,1,AL,1,Autauga County,All Permits,2003,283
8,1,AL,1,Autauga County,All Permits,2002,276
9,1,AL,1,Autauga County,All Permits,2001,400


In [8]:
state_permits = pd.DataFrame(permits.groupby(['StateAbbr', 'year', 'variable'])['value'].sum()).reset_index().rename(columns = {'StateAbbr':'state'})
state_permits

,state,year,variable,value
0,AK,1980,2-Unit Multifamily,174
1,AK,1980,3 & 4-Unit Multifamily,76
2,AK,1980,5+-Unit Multifamily,250
3,AK,1980,All Multifamily,500
4,AK,1980,All Permits,1837
5,AK,1980,Single Family,1337
6,AK,1981,2-Unit Multifamily,386
7,AK,1981,3 & 4-Unit Multifamily,228
8,AK,1981,5+-Unit Multifamily,629
9,AK,1981,All Multifamily,1243


In [9]:
state_geo = pd.DataFrame(states).transpose().reset_index().rename(columns = {'index':'state'})
state_geo

,state,lats,lons,name,region
0,AK,"[56.09688, 55.91078, 55.26456, 54.69554, 55.21...","[-130.24554, -130.01763, -130.00173, -131.2168...",Alaska,Northwest
1,AL,"[30.99763, 30.94243, 30.89199, 30.85923, 30.83...","[-87.58552, -87.59581, -87.62123, -87.6272, -8...",Alabama,South
2,AR,"[33.56679, 33.56656, 33.55479, 33.55887, 33.57...","[-94.26958, -94.30425, -94.34879, -94.40149, -...",Arkansas,Central
3,AZ,"[34.87057, 35.00186, 35.00332, 35.07971, 35.11...","[-114.63332, -114.63349, -114.63423, -114.6089...",Arizona,Southwest
4,CA,"[37.77205, 37.77078, 37.76913, 37.76387, 37.75...","[-123.00111, -122.99754, -122.99509, -122.9874...",California,Southwest
5,CO,"[38.215, 38.40118, 38.60929, 38.81393, 38.9578...","[-109.04984, -109.06017, -109.06015, -109.0565...",Colorado,Southwest
6,CT,"[42.0324, 42.025, 42.03006, 42.03398, 42.03198...","[-72.64313, -72.60487, -72.57337, -72.45609, -...",Connecticut,Northeast
7,DC,"[38.97872, 38.98378, 38.96493, 38.95822, 38.95...","[-77.06276, -77.02561, -77.00141, -76.99288, -...",District of Columbia,Mid-Atlantic
8,DE,"[39.63895, 39.68084, 39.72204, 39.72221, 39.72...","[-75.7878, -75.78909, -75.78861, -75.78861, -7...",Delaware,Mid-Atlantic
9,FL,"[24.72148, 24.72333, 24.72623, 24.72628, 24.70...","[-82.88318, -82.87484, -82.86562, -82.80018, -...",Florida,Southeast


In [10]:
state_permits_geo = pd.merge(state_permits, state_geo, on='state')
state_permits_geo

,state,year,variable,value,lats,lons,name,region
0,AK,1980,2-Unit Multifamily,174,"[56.09688, 55.91078, 55.26456, 54.69554, 55.21...","[-130.24554, -130.01763, -130.00173, -131.2168...",Alaska,Northwest
1,AK,1980,3 & 4-Unit Multifamily,76,"[56.09688, 55.91078, 55.26456, 54.69554, 55.21...","[-130.24554, -130.01763, -130.00173, -131.2168...",Alaska,Northwest
2,AK,1980,5+-Unit Multifamily,250,"[56.09688, 55.91078, 55.26456, 54.69554, 55.21...","[-130.24554, -130.01763, -130.00173, -131.2168...",Alaska,Northwest
3,AK,1980,All Multifamily,500,"[56.09688, 55.91078, 55.26456, 54.69554, 55.21...","[-130.24554, -130.01763, -130.00173, -131.2168...",Alaska,Northwest
4,AK,1980,All Permits,1837,"[56.09688, 55.91078, 55.26456, 54.69554, 55.21...","[-130.24554, -130.01763, -130.00173, -131.2168...",Alaska,Northwest
5,AK,1980,Single Family,1337,"[56.09688, 55.91078, 55.26456, 54.69554, 55.21...","[-130.24554, -130.01763, -130.00173, -131.2168...",Alaska,Northwest
6,AK,1981,2-Unit Multifamily,386,"[56.09688, 55.91078, 55.26456, 54.69554, 55.21...","[-130.24554, -130.01763, -130.00173, -131.2168...",Alaska,Northwest
7,AK,1981,3 & 4-Unit Multifamily,228,"[56.09688, 55.91078, 55.26456, 54.69554, 55.21...","[-130.24554, -130.01763, -130.00173, -131.2168...",Alaska,Northwest
8,AK,1981,5+-Unit Multifamily,629,"[56.09688, 55.91078, 55.26456, 54.69554, 55.21...","[-130.24554, -130.01763, -130.00173, -131.2168...",Alaska,Northwest
9,AK,1981,All Multifamily,1243,"[56.09688, 55.91078, 55.26456, 54.69554, 55.21...","[-130.24554, -130.01763, -130.00173, -131.2168...",Alaska,Northwest


In [53]:
mo_permits = permits[(permits['StateAbbr'] == 'MO') & 
                     (permits['variable'] == 'All Permits') &
                     (permits['year'] >= 2005)].copy()
mo_permits_summed = pd.DataFrame(mo_permits.groupby('year')['value'].sum()).reset_index()

In [54]:
mo_timeseries = figure(plot_width=800, plot_height=500, title='Missouri Building Permits')
mo_timeseries.xaxis.axis_label = 'Year'
mo_timeseries.yaxis.axis_label = 'Number of Permits'
mo_timeseries.title.text_font_size = '20pt'

mo_timeseries.line(mo_permits_summed['year'], mo_permits_summed['value'],
                   line_width=4, line_color='red')

show(mo_timeseries)

In [64]:
mo_permits_pivoted = (mo_permits[(mo_permits['year'] == 2006) | 
                                (mo_permits['year'] == 2010)]
                      .pivot(index='countyname', columns='year', values='value')
                      .reset_index())
mo_permits_pivoted['delta'] = mo_permits_pivoted[2006] - mo_permits_pivoted[2010]
mo_permits_pivoted

year,countyname,2006,2010,delta
0,Adair County,45.0,17.0,28.0
1,Andrew County,19.0,6.0,13.0
2,Atchison County,1.0,1.0,0.0
3,Audrain County,32.0,8.0,24.0
4,Barry County,80.0,22.0,58.0
5,Barton County,7.0,7.0,0.0
6,Bates County,23.0,22.0,1.0
7,Benton County,9.0,1.0,8.0
8,Bollinger County,NaN,1.0,NaN
9,Boone County,1932.0,607.0,1325.0


In [ ]:
from bokeh.palettes import RdYlBu11 as palette
from bokeh.models import LogColorMapper 

# Create the color mapper
color_mapper = LogColorMapper(palette=palette)

mo

In [66]:
data = gpd.read_file('../Class_Task_20/shp/County-AK-HI-Moved-USA-Map.shp')
data

,OBJECTID,FIPS,FIPSNum,StateFIPS,CountyFIPS,StateFIPSN,CntyFIPSN,StateName,CntyName,SEERArea,CensusReg,CensusDiv,XCentroid,YCentroid,Notes,geometry
0,1,02900,2900,02,900,2,900,Alaska,(entire state),Alaska,West,Pacific,-1.882092e+06,2.310348e+06,,(POLYGON ((-1447392.184973718 -1440681.7663283...
1,9,53073,53073,53,073,53,73,Washington,Whatcom County,Seattle (Puget Sound),West,Pacific,-1.886823e+06,1.519337e+06,,(POLYGON ((-1979920.976470158 1565781.66668992...
2,10,23003,23003,23,003,23,3,Maine,Aroostook County,N/A (remainder of the U.S.),Northeast,New England,2.075817e+06,1.324792e+06,,"POLYGON ((2067491.423331289 1291663.317634629,..."
3,11,27077,27077,27,077,27,77,Minnesota,Lake of the Woods County,N/A (remainder of the U.S.),Midwest,West North Central,8.124696e+04,1.256649e+06,,"POLYGON ((50161.36833250523 1281479.096953144,..."
4,12,23021,23021,23,021,23,21,Maine,Piscataquis County,N/A (remainder of the U.S.),Northeast,New England,2.050567e+06,1.221412e+06,,"POLYGON ((2067491.423331289 1291663.317634629,..."
5,13,23025,23025,23,025,23,25,Maine,Somerset County,N/A (remainder of the U.S.),Northeast,New England,2.009509e+06,1.173594e+06,,"POLYGON ((1995299.652640056 1291361.254018348,..."
6,14,27135,27135,27,135,27,135,Minnesota,Roseau County,N/A (remainder of the U.S.),Midwest,West North Central,1.394637e+04,1.256717e+06,,"POLYGON ((50161.36833250523 1281479.096953144,..."
7,15,27071,27071,27,071,27,71,Minnesota,Koochiching County,N/A (remainder of the U.S.),Midwest,West North Central,1.660829e+05,1.200338e+06,,"POLYGON ((117557.6978672724 1212827.089465206,..."
8,16,27137,27137,27,137,27,137,Minnesota,St. Louis County,N/A (remainder of the U.S.),Midwest,West North Central,2.671978e+05,1.132846e+06,,"POLYGON ((216928.7960718181 1243824.011862075,..."
9,17,27007,27007,27,007,27,7,Minnesota,Beltrami County,N/A (remainder of the U.S.),Midwest,West North Central,8.010743e+04,1.169107e+06,,"POLYGON ((49058.49575547688 1231055.723082185,..."


In [74]:
source = ColumnDataSource(state_geo)

bob = figure()
bob.patches('lons', 'lats', source=source)

show(bob)

ValueError: Out of range float values are not JSON compliant